In [20]:
import pandas as pd 

import string
from nltk.corpus import stopwords

In [8]:
#read data from csv file 
df_imdb = pd.read_csv('imdb_labelled.txt', sep='\t', header=None, names = ['text', 'label'])
df_imdb.head()

,text,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [9]:
#view data statistics using describe()
df_imdb.describe()

,label
count,748.000000
mean,0.516043
std,0.500077
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [16]:
df_imdb.groupby('label').describe()

length                                                          
       count        mean         std  min    25%   50%     75%     max
label                                                                 
0      362.0  117.185083  489.595391  8.0  42.00  68.0  104.00  7944.0
1      386.0  105.207254  237.312394  7.0  43.25  77.0  123.75  4488.0

In [14]:
# Verify the length of the messages and add it as a new column,
df_imdb['length'] = df_imdb['text'].apply(len)
df_imdb.head()

,text,label,length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [26]:
def removeStopWords(sentence):
    no_punctuation = [char for char in sentence if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    words_list = no_punctuation.split()
    clean_words = [word for word in words_list if word not in stopwords.words('english')]
    return clean_words

In [27]:
df_imdb['text'].head().apply(removeStopWords)

0    [A, slowmoving, aimless, movie, distressed, dr...
1    [Not, sure, lost, flat, characters, audience, ...
2    [Attempting, artiness, black, white, clever, c...
3               [Very, little, music, anything, speak]
4    [The, best, scene, movie, Gerardo, trying, fin...
Name: text, dtype: object

In [34]:
#Apply a transformer and fit the data in the bag of words,
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer(analyzer=removeStopWords)
tf_idf_transformer = TfidfTransformer()

bag_of_words_transformer = vectorizer.fit(df_imdb['text'])
bag_of_words = bag_of_words_transformer.transform(df_imdb['text'])

tfidf_bag_of_words = tf_idf_transformer.fit(bag_of_words)
tfidf_message = tfidf_bag_of_words.transform(bag_of_words)

In [35]:
#Print the shape for the transformer, and
tfidf_message.shape

(748, 3340)

In [41]:
# Naive Bayes model
from sklearn.naive_bayes import MultinomialNB

spam_detect_model = MultinomialNB().fit(tfidf_message,df_imdb['label'])

In [47]:
#Check the model for predicted and expected values.

n = 9

predicted_raw = df_imdb['text'][n]
bag_pred_tf = bag_of_words_transformer.transform([predicted_raw])
tfidf_pred = tfidf_bag_of_words.transform(bag_pred_tf)
predicted = spam_detect_model.predict(tfidf_pred)

expected = df_imdb['label'][n]

print('expected', expected)
print('predicted', predicted)

expected 1
predicted [1]
